This example shows a basic example of "Feff fitting" with larch.  

We start with some import statements, then 
   1. read in data for ZnSe, Zn K edge
   2. do background subtraction and plot $\chi(k)$
   3. set the Fourier transform ranges
   4. define a Group of Parameters for the fit
   5. read in a Feff Path for Zn-Se 
   6. run the fit and view the results

In [3]:
# step 0: imports
import numpy as np
from larch.xafs import (pre_edge, autobk, xftf, 
                        ff2chi, feffpath, feffit_transform, feffit_dataset, feffit, feffit_report)
from larch.fitting import param, guess, param_group
from larch.io import read_ascii
from larch.plot.plotly_xafsplots import (plot_mu, plot_chik, plot_chir, plot_chifit, 
                                         multi_plot, plotlabels, set_label_weight)

In [4]:
# step 1: read in the data and make mu(E)

data  = read_ascii('../xafsdata/znse_zn_xafs.001', labels=['energy', 'time', 'i0', 'itrans'])
data.mu = -np.log(data.itrans/data.i0)
pre_edge(data)
fig1 = plot_mu(data, show_norm=True)

In [5]:
# step 2: do and check the background subtraction
autobk(data, rbkg=1.0, kw=2)
fig2 = plot_chik(data, kweight=2)

In [6]:
# step 3: set transform / fit ranges
trans = feffit_transform(kmin=3, kmax=12, kweight=2, dk=4, window='kaiser', rmin=1.0, rmax=3.0)
xftf(data, kmin=3, kmax=12, kweight=2, dk=4, window='kaiser')
fig3 = plot_chir(data)

In [7]:
# step 4: setup fitting Parameters: we will vary 
#   amplitude, e0, sigma2, and delta_R

pars = param_group(amp    = param(1.0, vary=True),
                   del_e0 = param(0.5, vary=True),
                   sig2   = param(0.005, vary=True),
                   del_r  = guess(0.001, vary=True))

In [8]:
# step 5: define a Feff Path for Zn-Se scattering

# using a Path file from a previous Feff calculation for Zn-Se (shown elsewhere)
# here we give math expressions using the Parameters named in the Parameter group 
# to set the values for the Path Parameters in the EXAFS equation
path_znse = feffpath('../feffit/feff_znse.dat', 
                     s02='amp', e0='del_e0', sigma2='sig2', deltar='del_r')

tmpsum = ff2chi([path_znse], params=pars)
fig4 = plot_chik(tmpsum, kweight=2)
fit5 = multi_plot([{'xdata':tmpsum.k, 'ydata':tmpsum.chi*tmpsum.k**2, 'label': 'Feff'},
            {'xdata':data.k, 'ydata':data.chi*data.k**2, 
             'label': 'Data','xlabel': plotlabels.k, 
             'ylabel': set_label_weight(plotlabels.chikw, 2) }])


In [9]:
# step 6: run and view the fit

# now that we have data, a set of paths, and Fourier transform values, we can
# set up a "Feffit Dataset" and then fit that with the group of Parameters

# define dataset to include data, pathlist, transform
dset = feffit_dataset(data=data, pathlist=[path_znse], transform=trans)

# do the fit...
result = feffit(pars, dset)

# print out the fit report
print(feffit_report(result))

=================== FEFFIT RESULTS ====================
[[Statistics]]
   n_function_calls   = 26
   n_variables        = 4
   n_data_points      = 130
   n_independent      = 12.4591559
   chi_square         = 246.622347
   reduced chi_square = 29.1544865
   r-factor           = 0.00282983
   Akaike info crit   = 45.1955944
   Bayesian info crit = 47.2854175
 
[[Dataset]]
   unique_id          = 'djwb5nrk'
   fit space          = 'r'
   r-range            = 1.000, 3.000
   k-range            = 3.000, 12.000
   k window, dk       = 'kaiser', 4.000
   paths used in fit  = ['../feffit/feff_znse.dat']
   k-weight           = 2
   epsilon_k          = Array(mean=4.4808e-4, std=4.4430e-4)
   epsilon_r          = 0.0063023
   n_independent      = 12.459
 
[[Variables]]
   amp            =  0.9938163 +/- 0.0547348   (init=  1.0000000)
   del_e0         =  5.8934172 +/- 0.6999331   (init=  0.5000000)
   del_r          = -0.0012320 +/- 0.0034247   (init=  1.0000e-3)
   sig2           =  0.00654

In [10]:
# finally, plot the results
fig6, fig7 = plot_chifit(result.datasets[0], show_real=True)